<a href="https://colab.research.google.com/github/luisosmx/Python_exercises/blob/main/class_ExtractLoad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-storage
!pip install pandas google-cloud-storage

In [23]:
from google.cloud import storage
from google.cloud import bigquery
import os
import pandas as pd


class ExtractLoad:

  def __init__(self, bucket_name, object_name, destination, path_json, all_column_names, filter_column, path_json_load, table_id, schema):
    self.bucket_name = bucket_name
    self.object_name = object_name
    self.destination = destination
    self.path_json = path_json
    self.all_column_names = all_column_names
    self.filter_column = filter_column
    self.path_json_load = path_json_load
    self.table_id = table_id
    self.schema = schema

  def download_file(self):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = self.path_json
    storage_client = storage.Client()
    bucket = storage_client.bucket(self.bucket_name)
    blob = bucket.blob(self.object_name)
    #Descarga el contenido del objeto como bytes
    blob.download_to_filename(self.destination)
    print("Archivo descargado como 'file.csv")

  def clean_file(self):
    df = pd.read_csv(destination, encoding='latin-1')
    column_names = df.columns.tolist()
    filtered_column_names = [col for col in self.all_column_names if col in self.filter_column]
    self.filtered_df = df.loc[:, filtered_column_names]
    print('DataFrame is clean')
    return self.filtered_df

  def load_file(self):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_json_load
    client = bigquery.Client()
    job_config = bigquery.LoadJobConfig(schema = schema, write_disposition="WRITE_TRUNCATE")
    job = client.load_table_from_dataframe(self.filtered_df, table_id, job_config=job_config)
    job.result()
    table = client.get_table(table_id)
    print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))


In [ ]:
bucket_name = 'bucketdata_storage'
object_name = '/content/inah_visitantes_zonas_general_anual_2023.csv'
destination = '/content/file.csv'
path_json = '/content/path.json'
all_column_names = ['Estado', 'Clave SIINAH', 'Tipo', 'Centro INAH', 'enero', 'Unnamed: 5', 'febrero', 'Unnamed: 7', 'marzo', 'Unnamed: 9', 'abril', 'Unnamed: 11', 'mayo', 'Unnamed: 13', 'junio', 'Unnamed: 15', 'julio', 'Unnamed: 17']
filter_column = ['estado', 'tipo', 'centro_inah', 'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio']
path_json_load = '/content/path.json'
table_id = 'data-project-395314.data_project.inah_visitantes'
schema = schema=[
                bigquery.SchemaField("estado", "STRING"),
                bigquery.SchemaField("tipo", "STRING"),
                bigquery.SchemaField("centro_inah", "STRING"),
                bigquery.SchemaField("enero", "STRING"),
                bigquery.SchemaField("febrero", "STRING"),
                bigquery.SchemaField("marzo", "STRING"),
                bigquery.SchemaField("abril", "STRING"),
                bigquery.SchemaField("mayo", "STRING"),
                bigquery.SchemaField("junio", "STRING"),
                bigquery.SchemaField("julio", "STRING"),
                ]

extract_load = ExtractLoad(bucket_name,
                           object_name,
                           destination,
                           path_json,
                           all_column_names,
                           filter_column,
                           path_json_load,
                           table_id,
                           schema)

extract_load.download_file()
extract_load.clean_file()
extract_load.load_file()